# Computational Analysis of Sounds and Music (CH-CASM-M)

## 02 - Fundamentals of Audio Processing (2/2)

**WS 2025/2026**

Prof. Dr. Jakob Abeßer, jakob.abesser@uni-bamberg.de

Last update: 28.10.2025

**Outline**

In this notebook, you will learn 
 - how to compute and visualize the STFT, Mel Spectrogram, and CQT
 - how to compute a chromagramm
 - how to compute the MFCC features

In [ ]:
!pip install wget

In [ ]:
import numpy as np
import wget
import os
import matplotlib
import librosa
%matplotlib inline
import matplotlib.pyplot as pl
import IPython.display as ipd

### Get audio example files

This script loads 2 audio files that we need here.

In [ ]:
if not os.path.isfile('piano.wav') or not os.path.isfile('bird.wav') or not os.path.isfile('c_maj_inv_2_sounds.wav'):
    for fn in ('piano.wav', 'bird.wav', 'c_maj_inv_2_sounds.wav'):
        wget.download('https://github.com/CHBamberg/CH-CASM-M-2025/raw/refs/heads/main/data/{}'.format(fn), 
                      out=fn, bar=None)
else:
    print('Files already exist!')


### Waveform Visualization

Let's plot our waveform

In [ ]:
# (1) use the sample rate of the file, load stereo if needed
fs_fix = 44100
x, fs = librosa.load('bird.wav', sr=fs_fix)  # in this case, the signal is upsampled to a higher sample rate

In [ ]:
number_of_samples = len(x)
seconds_per_sample = 1/fs
frames_in_seconds = np.arange(number_of_samples)*seconds_per_sample

In [ ]:
pl.figure(figsize=(14,3))
pl.plot(frames_in_seconds, x)
pl.xlabel('Time [s]')
pl.ylabel('Amplitude')
pl.show()

### Spectrogram using Short-Time Fourier Transform (STFT)

Let's first check the librosa documentation: https://librosa.org/doc/main/generated/librosa.stft.html?highlight=stft#librosa-stft

The most important parameters are 
  - **win_length** - this is the size of our analysis window (in samples)
  - **hop_length** - this is the hop size of our analysis window (in samples), usually this is chosen to be half the window size
  - (**n_fft**) - this is the used "FFT size", which can be bigger than the **win_length** (but should be a power of two, such that the Fast Fourier Transform (FFT) algorithm can be used internally)
  

In [ ]:
# Compute the STFT
n_fft = 2048
hop_length = 1024
X = librosa.stft(x, n_fft=n_fft, hop_length=hop_length)  # using the default values: n_fft=2048, 
print("Shape of STFT:", X.shape)  # we get n_fft//2 - 1 bins, the reason is that the STFT has a 
                                  # symmetric structure and we can discard several entries
print("Data type of STFT:", X.dtype)  # ! the STFT is complex and has a magnitude and a phase

# We'll focus on the magnitude of the STFT
S = np.abs(X)
print("Shape of the magnitude spectrogram:", S.shape)
print("Data type of the magnitude spectrogram:", S.dtype)  # ok

In [ ]:
# let's plot it
pl.figure(figsize=(12,6))
pl.imshow(S, aspect="auto", cmap="Greys")
pl.colorbar()
pl.show()

There a several **problems** with this plot:
  1. the frequency axis is flipped (lower frequencies are shown on top and higher frequencies are shown at the bottom)
  2. only the loudest frequency components are visible  
  3. we want the axes to show the frequency in Hz (y-axis) and the time in seconds (x-axis), at the moment, we only see the frequency bin (y-axis) of the STFT and the frame number (x-axis)


In [ ]:
# Issue 1.) use "origin" parameter for imshow()
pl.figure(figsize=(12,6))
# TASK: we need one additional parameter in the imshow command from before to vertically flip the image
pl.colorbar()
pl.show()

In [ ]:
# Issue 2.) apply logarithmic compression to the magnitude values -> this converts the linear magnitudes to decibels (dB)
S_dB = None # TASK check for a suitable function from librosa to implement the logarithmic scaling

In [ ]:
fig = pl.figure(figsize=(12,6))
pl.imshow(S_dB, aspect="auto", origin="lower", cmap="Greys")
pl.colorbar(format='%+2.0f dB')
pl.show()

**Observation**
1.  also parts with lower magnitudes are now better visible
2.  the value range shifts from [0, 70] to [-80, 0], this is because of the logarithmic compression

In [ ]:
# Issue 3.) define maximum frequency, and maximum time value
f_max = fs/2  # Nyquist frequency
t_max = number_of_samples / fs
print(f'Upper time limit = {f_max} s')
print(f'Upper frequency limit = {t_max} Hz')

In [ ]:
pl.figure(figsize=(6,4))
# TASK add another parameter of imshow to make use of the upper time and frequency limits in our plot
pl.xlabel('Time [s]')
pl.ylabel('Frequency [Hz]')
pl.colorbar(format='%+2.0f dB')
pl.show()

**Practical Alternative**

As an alternative, we can use the build-in visualization function ```specshow``` of librosa, which allows to visualize STFT spectrograms, Mel spectrograms, and others.

Check the documentation for more info: https://librosa.org/doc/main/generated/librosa.display.specshow.html

In [ ]:
from librosa.display import specshow

fig, ax = pl.subplots()
# note the keyword "mel", which indicates that a mel frequency axis is used
img = specshow(S_dB, x_axis='time', y_axis='hz', sr=fs, ax=ax, cmap="Greys")
fig.colorbar(img, ax=ax, format='%+2.0f dB')
ax.set(title='STFT Magnitude Spectrogram (specshow)')
pl.show()

### Mel-Spectrogram

Let's first check the librosa documentation: https://librosa.org/doc/main/generated/librosa.feature.melspectrogram.html

The most important parameters are 
  - **y** - audio sample vector ($x$)
  - **sr** - sampling rate of the audio signal (in Hz)
  - **n_mels** - number of Mel frequency bands (commonly: 64 or 128)
  - **win_length** - see above (STFT)
  - **hop_length** - see above (STFT)
  - **n_fft** - see above (STFT)
  

In [ ]:
M = librosa.feature.melspectrogram(y=x, n_fft=2048, hop_length=1024, n_mels=128)  
print("Shape of Mel spectrogram:", M.shape)  # frequency x time: we get n_mels frequency bands

Let's visualize it. We want to
1. convert the time axis (horizontal axis) to seconds (as done before for the STFT)
2. have physical frequency values [Hz] at the vertical axis, that correspond to the 128 mel bands 

In [ ]:
# apply dB compression as before
M_dB = None # TASK use the right librosa function for logarithmic magnitude scaling

In [ ]:
fig, ax = pl.subplots()
# note the keyword "mel", which indicates that a mel frequency axis is used
img = librosa.display.specshow(M_dB, x_axis='time', y_axis='mel', sr=fs, ax=ax, cmap='viridis')
fig.colorbar(img, ax=ax, format='%+2.0f dB')
ax.set(title='Mel-frequency spectrogram')
pl.show()

### Constant-Q Transform

Let's first check the librosa documentation: https://librosa.org/doc/main/generated/librosa.cqt.html

The most important parameters are 
  - **y** - audio sample vector ($x$)
  - **sr** - sampling rate of the audio signal (in Hz)
  - **hop_length** - see above (STFT)
  - **f_min** - minimum frequency (we can use the default value of 32.70 Hz which corresponds to the note C1)
  - **n_bins** - total number of frequency bins (e.g., for a frequency resolution of one bin per semitone and 4 octaves, this would be 4 * 12 = 48)
  - **bins_per_octave** - Logarithmic frequency resolution (frequency bins per octave, commonly: 12 or 36)
  - **tuning** - Tuning offset (can be used if known tuning frequency of an audio recording deviates from 440 Hz) 
  

We'll now use a short piano recording as running example.

In [ ]:
fn_wav = os.path.join(dir_wav, 'piano.wav')  # original filename: 196765__xserra__piano-phrase.wav
x, fs = librosa.load(fn_wav)
ipd.display(ipd.Audio(data=x, rate=fs))

Let's compute the CQT for a frequency range of 5 octaves with a resolution of 1 bin per semitone (=12 bins per octave)

In [ ]:
n_octaves = 6  # let's capture 5 octaves starting from C1
bins_per_octave = 12  # let's choose a frequency resolution of 100 cent (= one frequency bin per semitone)
C = np.abs(librosa.cqt(x, sr=fs, 
                       fmin=2*32.70, # C 2 as lower frequency limit
                       n_bins=n_octaves*bins_per_octave , bins_per_octave=bins_per_octave))
print("Shape of CQT:", C.shape)  # logically, we get 60 frequency bins

In [ ]:
# dB magnitude scaling
C = librosa.amplitude_to_db(C)

In [ ]:
# we can use again the visualization tool provided by librosa
fig, ax = pl.subplots()
img = librosa.display.specshow(C, sr=fs, x_axis='time', y_axis='cqt_note', ax=ax, 
                       fmin=2*32.70)
fig.colorbar(img, ax=ax, format="%+2.0f dB")
pl.show()

### CENS (Chroma Energy Normalize) Chroma Features

Let's first check the librosa documentation: https://librosa.org/doc/main/generated/librosa.feature.chroma_cens.html

The most important parameters are 
  - **y** - audio sample vector ($x$)
  - **sr** - sampling rate of the audio signal (in Hz)
  - **hop_length** - see above (STFT)
  - **f_min** - minimum frequency (we can use the default value of 32.70 Hz which corresponds to the note C1)
  - **n_bins** - total number of frequency bins (e.g., for a frequency resolution of one bin per semitone and 4 octaves, this would be 4 * 12 = 48)
  - **bins_per_octave** - Logarithmic frequency resolution (frequency bins per octave, commonly: 12 or 36)
  - **tuning** - Tuning offset (can be used if known tuning frequency of an audio recording deviates from 440 Hz) 
  

In [ ]:
hop_length=128
cens = librosa.feature.chroma_cens(y=x, sr=fs, hop_length=hop_length, fmin=None, tuning=None, n_chroma=12, n_octaves=7, bins_per_octave=36)
t_max = len(x) / fs

pl.figure(figsize=(8,3))
pl.imshow(cens, aspect="auto", interpolation="None", origin="lower", extent=[0, t_max, 0, cens.shape[0]])
# let's create an interpretable pitch axis
pl.yticks([0.5, 2.5, 4.5, 5.5, 7.5, 9.5, 11.5], ['C', 'D', 'E', 'F', 'G', 'A', 'B'])
pl.xlabel('Time (seconds)')
pl.ylabel('Pitch Class')
pl.title('CENS')
pl.colorbar()
pl.tight_layout()
pl.show()

The chroma features give a nice visual summary of the pitch class distribution over time. 

**Note**: this is not a transcription as the pitch classes of the partial frequencies also affect this representation!

### Chord Inversions

Now we'll analyze a new file. It has 4 successive chords
  - C major (root position): C E G
  - C/E (first inversion): E G C
  - C/G (second inversion): G C E
  - C major (root position, one octave higher)
  
The chord sequence is first played using a piano, then repeated using a synthesizer sound.

Here's the pianoroll view:

In [ ]:
image_url = "https://github.com/CHBamberg/CH-CASM-M-2025/raw/refs/heads/main/data/c_maj_inv_2_sounds.png?raw=true"
ipd.display(ipd.Image(url=image_url))

Let's listen to the audio file

In [ ]:
fn_wav = 'c_maj_inv_2_sounds.wav'
x, fs = librosa.load(fn_wav)
ipd.display(ipd.Audio(data=x, rate=fs))

Now, let's have a look to the CENS Chromagram!

In [ ]:
hop_length=128
cens = librosa.feature.chroma_cens(y=x, sr=fs, hop_length=hop_length, fmin=None, tuning=None, n_chroma=12, n_octaves=7, bins_per_octave=36)
t_max = len(x) / fs

pl.figure()
pl.imshow(cens, aspect="auto", interpolation="None", origin="lower", extent=[0, t_max, 0, cens.shape[0]])
# let's create an interpretable pitch axis
pl.yticks([0.5, 2.5, 4.5, 5.5, 7.5, 9.5, 11.5], ['C', 'D', 'E', 'F', 'G', 'A', 'B'])
pl.xlabel('Time (seconds)')
pl.ylabel('Pitch Class')
pl.title('CENS')
pl.tight_layout()
pl.show()

What do you observe?

1) Why is the chromagram pattern not changing for different inversions?
2) Why is the chromagram pattern not changing for different instrument timbres?
3) Can you guess where the "transient" (vertical) structures come frome?

For comparison, let's check the CQT again, here we should see how the fundamental frequency and the harmonics are increasing in frequency along the chord sequence (similar to the piano roll)

In [ ]:
n_octaves = 7  # let's capture 7 octaves starting from C1
bins_per_octave = 12  # let's choose a frequency resolution of 100 cent (= one frequency bin per semitone)
C = np.abs(librosa.cqt(x, sr=fs, n_bins=n_octaves*bins_per_octave , bins_per_octave=bins_per_octave))
C = librosa.amplitude_to_db(C)  # dB magnitude scaling
fig, ax = pl.subplots()
img = librosa.display.specshow(C, sr=fs, x_axis='time', y_axis='cqt_note', ax=ax)
fig.colorbar(img, ax=ax, format="%+2.0f dB")
pl.show()

### Mel-Frequency Cepstral Coefficients (MFCC)

Let's first check the librosa documentation: https://librosa.org/doc/main/generated/librosa.cqt.html

The most important parameters are 
  - **y** - audio sample vector ($x$)
  - **sr** - sampling rate of the audio signal (in Hz)
  - **n_fft** - see above (STFT)
  - **hop_length** - see above (STFT)
  - **n_mfcc** - Number of MFC coefficients  

In [ ]:
# let's compute and visualize 13 MFCCs
n_mfcc = 20
n_fft = 2048
hop_length = 1024
mf = librosa.feature.mfcc(y=x, sr=fs, S=None, n_mfcc=n_mfcc, dct_type=2, norm='ortho', lifter=0, n_fft=n_fft, hop_length=hop_length)
pl.figure()
pl.imshow(mf, aspect="auto", origin="lower", extent=[0, t_max, 0, mf.shape[0]])
pl.xlabel('Time [s]')
pl.ylabel('MFCC')
pl.show()

We will use the MFCC features in the upcoming seminars as timbre feature for different classification tasks ...

Done :)